# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,-6.43,74,100,5.69,SJ,1712779143
1,1,st. john's,47.5649,-52.7093,1.27,73,100,5.66,CA,1712778947
2,2,west island,-12.1568,96.8225,25.99,78,40,3.09,CC,1712779144
3,3,iqaluit,63.7506,-68.5145,-1.15,68,20,2.06,CA,1712779145
4,4,kishanganj,26.1167,87.9333,24.92,30,0,3.74,IN,1712779146


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_city_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.3,
    color = "City"
)

# Display the map
map_city_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp']<=21) & (city_data_df['Max Temp']>=0)\
                            & (city_data_df['Humidity']<70) & (city_data_df['Humidity']>=10),:]

# Drop any rows with null values
ideal_city_df.dropna(how="any")

# Display sample data
ideal_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,lebu,-37.6167,-73.6500,14.96,56,100,3.68,CL,1712779082
11,11,puerto natales,-51.7236,-72.4875,10.25,50,40,7.72,CL,1712779151
23,23,constantia,44.1833,28.6500,10.39,68,0,2.90,RO,1712779160
28,28,chibougamau,49.9168,-74.3659,9.79,50,75,6.17,CA,1712779102
30,30,hotan,37.0997,79.9269,15.63,15,42,1.29,CN,1712779165
37,37,ghormach,35.7306,63.7826,16.00,49,0,2.50,AF,1712779169
42,42,happy valley-goose bay,53.3168,-60.3315,4.17,48,20,1.03,CA,1712779173
44,44,badger,64.8000,-147.5333,3.17,57,20,0.45,US,1712779174
69,69,bethel,41.3712,-73.4140,19.47,57,40,2.06,US,1712778964
70,70,bismarck,46.8083,-100.7837,10.91,61,100,12.35,US,1712779000


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
6,lebu,CL,-37.6167,-73.6500,56,
11,puerto natales,CL,-51.7236,-72.4875,50,
23,constantia,RO,44.1833,28.6500,68,
28,chibougamau,CA,49.9168,-74.3659,50,
30,hotan,CN,37.0997,79.9269,15,
37,ghormach,AF,35.7306,63.7826,49,
42,happy valley-goose bay,CA,53.3168,-60.3315,48,
44,badger,US,64.8000,-147.5333,57,
69,bethel,US,41.3712,-73.4140,57,
70,bismarck,US,46.8083,-100.7837,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation.hotel',
    "filter":'',
    "limit":5,
    "bias":'',
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    long = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
lebu - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
constantia - nearest hotel: Hotel Chérica
chibougamau - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
ghormach - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
badger - nearest hotel: Clarion Hotel & Suites
bethel - nearest hotel: Hampton Inn Danbury
bismarck - nearest hotel: Radisson Hotel Bismarck
thompson - nearest hotel: Thompson Inn
yanchep - nearest hotel: Yanchep Inn
bredasdorp - nearest hotel: Victoria Hotel
levuka - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
alausi - nearest hotel: LA CASITA Guest House
castillos - nearest hotel: A mi gente
las heras - nearest hotel: Posadas del Sur
jamestown - nearest hotel: DoubleTree Jamestown
hami - nearest hotel: Хами
norrtalje - nearest hotel: Åtellet
labrador city - nearest hotel: Carol Inn
cey

,City,Country,Lat,Lng,Humidity,Hotel Name
6,lebu,CL,-37.6167,-73.6500,56,No hotel found
11,puerto natales,CL,-51.7236,-72.4875,50,Dorotea Patagonia Hostel
23,constantia,RO,44.1833,28.6500,68,Hotel Chérica
28,chibougamau,CA,49.9168,-74.3659,50,No hotel found
30,hotan,CN,37.0997,79.9269,15,Hua Yi International Hotel (accepts foreigners)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.3,
    color = "City",
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)